# test for proper rotation of detector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from detector import make_detector, rotate_about_normal, rotate_about_horizontal, rotate_about_vertical, intersect_detector
%matplotlib widget

In [ ]:
det_pixels = (200,200) #horizontal, vertical
det_qs = (8,5) #horizontal, vertical (these are absolute maximums. detector centered at 0)
det_x_init, det_y_init, det_z_init, det_h, det_v = make_detector(det_qs[0], det_pixels[0], det_qs[1], det_pixels[1])

psi = 90 #rotation in degrees of detector about detector normal axis
det_x, det_y, det_z = rotate_about_normal(det_x_init, det_y_init, det_z_init, psi)
phi = 90 #rotation in degrees of detector about detector vertical axis
det_x, det_y, det_z = rotate_about_vertical(det_x, det_y, det_z, phi)
theta = 0 #rotation in degrees of detector about detector horizontal axis
det_x, det_y, det_z = rotate_about_horizontal(det_x, det_y, det_z, theta)

psi = 90 #rotation in degrees of detector about detector normal axis
det_x, det_y, det_z = rotate_about_normal(det_x, det_y, det_z, psi)
phi = 0 #rotation in degrees of detector about detector vertical axis
det_x, det_y, det_z = rotate_about_vertical(det_x, det_y, det_z, phi)

In [ ]:
# Plot the original and rotated rectangles in 3D space
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot original rectangle
ax.plot_surface(det_x_init, det_y_init, det_z_init, alpha=0.5, color='blue')

# Plot rotated rectangle
ax.plot_surface(det_x, det_y, det_z, alpha=0.5, color='red')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('2D Rectangle Rotated about Z-axis and then new Y-axis in 3D Space')

# Setting equal aspect ratio
max_range = np.array([det_x_init.max()-det_x_init.min(), det_y_init.max()-det_y_init.min(), det_z_init.max()-det_z_init.min()]).max() / 2.0
mid_x = (det_x_init.max()+det_x_init.min()) * 0.5
mid_y = (det_y_init.max()+det_y_init.min()) * 0.5
mid_z = (det_z_init.max()+det_z_init.min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.show()

In [ ]:
# Parameters
h_max = 5
v_max = 3
num_pixels_h = 100
num_pixels_v = 60
phi = 90  # Rotation angle in degrees about the z-axis
theta = 45  # Rotation angle in degrees about the new y-axis

# Convert angles to radians
phi_rad = np.radians(phi)
theta_rad = np.radians(theta)

# Generate grid values
h_axis_vals = np.linspace(-h_max, h_max, num_pixels_h)
v_axis_vals = np.linspace(-v_max, v_max, num_pixels_v)
det_y_grid, det_z_grid = np.meshgrid(h_axis_vals, v_axis_vals)
det_x_grid = np.zeros_like(det_y_grid)

# Flatten the grids for easy manipulation
x_flat = det_x_grid.flatten()
y_flat = det_y_grid.flatten()
z_flat = det_z_grid.flatten()

# Create the rotation matrix for z-axis
R_z = np.array([
    [np.cos(phi_rad), -np.sin(phi_rad), 0],
    [np.sin(phi_rad),  np.cos(phi_rad), 0],
    [0,                0,               1]
])

# First rotation around the z-axis
coords = np.vstack([x_flat, y_flat, z_flat])
rotated_coords_z = R_z @ coords

# Reshape back to grid for easier manipulation
rotated_x_grid = rotated_coords_z[0].reshape(det_x_grid.shape)
rotated_y_grid = rotated_coords_z[1].reshape(det_y_grid.shape)
rotated_z_grid = rotated_coords_z[2].reshape(det_z_grid.shape)

# Step 2: Calculate the new rotation axis (new y-axis)
point1 = np.array([rotated_x_grid[0, 0], rotated_y_grid[0, 0], rotated_z_grid[0, 0]])
point2 = np.array([rotated_x_grid[0, -1], rotated_y_grid[0, -1], rotated_z_grid[0, -1]])
new_y_axis = point2 - point1
new_y_axis /= np.linalg.norm(new_y_axis)  # Normalize to get unit vector

# Step 3: Build the rotation matrix for ccw rotations about this unit vector
def rotation_matrix(axis, theta):
    axis = axis / np.linalg.norm(axis)
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    return np.array([[a*a + b*b - c*c - d*d, 2*(b*c + a*d), 2*(b*d - a*c)],
                     [2*(b*c - a*d), a*a + c*c - b*b - d*d, 2*(c*d + a*b)],
                     [2*(b*d + a*c), 2*(c*d - a*b), a*a + d*d - b*b - c*c]])

R_new_y = rotation_matrix(new_y_axis, theta_rad)

# Step 4: Apply this rotation matrix to our rotated coordinates
rotated_coords_final = R_new_y @ rotated_coords_z

# Reshape the final rotated coordinates back to the original shape
final_x_grid = rotated_coords_final[0].reshape(det_x_grid.shape)
final_y_grid = rotated_coords_final[1].reshape(det_y_grid.shape)
final_z_grid = rotated_coords_final[2].reshape(det_z_grid.shape)

# Plot the original and rotated rectangles in 3D space
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot original rectangle
ax.plot_surface(det_x_grid, det_y_grid, det_z_grid, alpha=0.5, color='blue')

# Plot rotated rectangle
ax.plot_surface(final_x_grid, final_y_grid, final_z_grid, alpha=0.5, color='red')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('2D Rectangle Rotated about Z-axis and then new Y-axis in 3D Space')

# Setting equal aspect ratio
max_range = np.array([det_x_grid.max()-det_x_grid.min(), det_y_grid.max()-det_y_grid.min(), det_z_grid.max()-det_z_grid.min()]).max() / 2.0
mid_x = (det_x_grid.max()+det_x_grid.min()) * 0.5
mid_y = (det_y_grid.max()+det_y_grid.min()) * 0.5
mid_z = (det_z_grid.max()+det_z_grid.min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.show()
